In [2]:
import numpy as np
import pandas as pd


from src.data_processing import (
    load_data_iso,
    load_data_cap,
    load_data_cap1,
)


In [3]:
patient_ids = [str(i) for i in range(102, 224)]
to_remove = [
    "103",
    "108",
    "110",
    "113",
    "115",
    "122",
    "126",
    "127",
    "128",
    "130",
    "134",
    "139",
    "143",
    "144",
    "146",
    "149",
    "152",
    "155",
    "156",
    "164",
    "174",
    "175",
    "176",
    "177",
    "180",
    "185",
    "186",
    "190",
    "193",
    "194",
    "197",
    "203",
    "206",
    "211",
    "212",
    "220",
]
patient_ids = list(set(patient_ids) - set(to_remove))
all_data_dict = {}

for patient_id in patient_ids:
    all_data = load_data_iso(patient_id)
    all_data_dict[patient_id] = all_data

patient_ids_cap = ["0620"]
patient_ids_cap1 =[
    "627",
    "0639",
    "0652",
    "0675",
    "CGM_007",
    "CGM_008",
    "CGM_009",
    "CGM_011",
    "CGM_013",
    "CGM_014",
    "CGM_017",
    "CGM_018",
    "CGM_020",
    "CGM_021",
    "CGM_022",
    "CGM_023",
    "CGM_024",
    "CGM_025",
]
for patient_id in patient_ids_cap:
    all_data = load_data_cap(patient_id)
    all_data_dict[patient_id] = all_data

for patient_id in patient_ids_cap1:
    all_data = load_data_cap1(patient_id)
    all_data_dict[patient_id] = all_data
    
patient_ids = patient_ids + patient_ids_cap + patient_ids_cap1

In [4]:
len(patient_ids)

105

In [5]:
# Output the results
for patient_id, df in all_data_dict.items():
    df['Tid'] = df['Tid'].dt.strftime('%d/%m/%Y %H:%M')
    all_data_dict[patient_id] = df

In [6]:
all_data_dict

{'187':        cbg               Tid  minutes_elapsed  days_elapsed
 0     22.0  06/01/2021 21:06                0      0.000000
 1     21.9  06/01/2021 21:11                5      0.003472
 2     21.7  06/01/2021 21:16               10      0.006944
 3     21.5  06/01/2021 21:21               15      0.010417
 4     21.4  06/01/2021 21:26               20      0.013889
 ...    ...               ...              ...           ...
 1311  20.9  11/01/2021 10:21             6555      4.552083
 1312  21.2  11/01/2021 10:26             6560      4.555556
 1313  21.5  11/01/2021 10:31             6565      4.559028
 1314  21.9  11/01/2021 10:36             6570      4.562500
 1315  22.1  11/01/2021 10:41             6575      4.565972
 
 [1316 rows x 4 columns],
 '125':       cbg               Tid  minutes_elapsed  days_elapsed
 0    12.5  09/11/2020 15:40                0      0.000000
 1    12.3  09/11/2020 15:45                5      0.003472
 2    12.0  09/11/2020 15:50               10 

In [14]:
import numpy as np
import pandas as pd

lower_threshold = 3.9  # mmol/L (lower bound for TIR)
upper_threshold = 10.0  # mmol/L (upper bound for TIR)

# Placeholder for glucose statistics
stats = {}

for patient_id, df in all_data_dict.items():
    # Extract glucose values (cbg column)
    glucose_values = df['cbg']
    
    # Calculate mean and standard deviation
    mean_glucose = np.nanmean(glucose_values)
    std_glucose = np.nanstd(glucose_values)
    
    # Calculate time in range, below range, and above range
    total_count = len(glucose_values)
    time_in_range = np.sum((glucose_values >= lower_threshold) & (glucose_values <= upper_threshold)) / total_count * 100
    time_below_range = np.sum(glucose_values < lower_threshold) / total_count * 100
    time_above_range = np.sum(glucose_values > upper_threshold) / total_count * 100
    
    # Store statistics
    stats[patient_id] = {
        "Mean": mean_glucose,
        "SD": std_glucose,
        "TIR (%)": time_in_range,
        "TBR (%)": time_below_range,
        "TAR (%)": time_above_range,
    }

# Convert stats dictionary to DataFrame and display
stats_df = pd.DataFrame(stats).T
# Calculate summary statistics (mean and std of all metrics)
summary_stats = stats_df.mean(axis=0).to_dict()
summary_stats_std = stats_df.std(axis=0).to_dict()

# Combine mean ± std for the summary row
summary_row = {
    metric: f"{summary_stats[metric]:.2f} ± {summary_stats_std[metric]:.2f}" 
    for metric in stats_df.columns
}
summary_row["Patient ID"] = "Overall"  # Add label for the summary row

# Append summary row to DataFrame
summary_df = pd.DataFrame([summary_row]).set_index("Patient ID")
stats_df = pd.concat([stats_df, summary_df])

# Display the final DataFrame
display(stats_df)


,Mean,SD,TIR (%),TBR (%),TAR (%)
187,14.43503,2.817158,6.990881,0.0,93.009119
125,10.73828,2.865778,41.935484,0.0,58.064516
105,8.447384,1.601381,85.586924,0.148588,13.670134
209,14.103841,2.789765,7.423372,0.0,92.337165
160,12.12043,3.331749,31.679074,0.0,68.320926
...,...,...,...,...,...
CGM_022,6.637967,1.43915,9.649261,0.0,0.023592
CGM_023,11.079344,3.924375,47.894103,0.0,51.143201
CGM_024,10.919471,2.79147,38.120567,0.0,55.673759
CGM_025,13.606193,4.355876,26.590909,0.0,72.5


In [15]:
# Function to check intervals and count days if consistent
def check_intervals_and_count_days(data_dict):
    results = {}
    for patient_id, df in data_dict.items():
        df_copy = df.copy()
        # Convert Tid column to datetime
        df_copy['Tid'] = pd.to_datetime(df_copy['Tid'], format='%d/%m/%Y %H:%M')
        
        # Calculate time differences in minutes
        time_diffs = df_copy['Tid'].diff().dropna().dt.total_seconds() / 60
        
        # Check if all intervals are 5 minutes
        consistent = (time_diffs <= 15).all()
        
        if consistent:
            # Calculate the number of days between the first and last timestamp
            total_days = (df_copy['Tid'].max() - df_copy['Tid'].min()).total_seconds() / (24 * 3600)
            results[patient_id] = total_days
        else:
            results[patient_id] = None  # Inconsistent intervals
    return results

# Check intervals for each patient
interval_and_days_results = check_intervals_and_count_days(all_data_dict)

# Output results
print(interval_and_days_results)

{'187': 4.565972222222222, '125': 1.6111111111111112, '105': 4.670833333333333, '209': 7.246527777777778, '160': 4.194444444444445, '147': 1.0694444444444444, '102': 2.767361111111111, '195': 5.722916666666666, '123': 4.584027777777778, '158': 8.253472222222221, '145': 1.7986111111111112, '214': 2.0347222222222223, '218': 1.9583333333333333, '215': 2.3506944444444446, '157': 1.3506944444444444, '208': 9.597222222222221, '178': 1.7638888888888888, '188': 6.361111111111111, '131': 7.653472222222222, '153': 3.8541666666666665, '196': 1.0659722222222223, '207': 1.9861111111111112, '189': 5.829861111111111, '116': 0.61875, '173': 6.569444444444445, '221': 2.0416666666666665, '120': 1.1215277777777777, '199': 2.3993055555555554, '201': 6.659722222222222, '200': 6.423611111111111, '140': 1.7534722222222223, '119': 1.65625, '117': 1.5381944444444444, '183': 1.3263888888888888, '184': 2.326388888888889, '213': 6.854166666666667, '210': 1.6527777777777777, '121': 9.51388888888889, '172': 5.72569

In [16]:
# Extract the values
values = list(interval_and_days_results.values())

# Calculate the mean
mean_value = np.mean(values)

# Calculate the standard deviation
std_value = np.std(values)

print(f"Mean: {mean_value}")
print(f"Standard Deviation: {std_value}")

Mean: 4.418472222222222
Standard Deviation: 4.496505134771695


In [17]:
# Extract all patient IDs with None as their result
inconsistent_patients = [patient_id for patient_id, days in interval_and_days_results.items() if days is None]

# Output the inconsistent patient IDs
print(len(inconsistent_patients))


0


In [18]:
all_data_dict

{'187':        cbg               Tid  minutes_elapsed  days_elapsed
 0     22.0  06/01/2021 21:06                0      0.000000
 1     21.9  06/01/2021 21:11                5      0.003472
 2     21.7  06/01/2021 21:16               10      0.006944
 3     21.5  06/01/2021 21:21               15      0.010417
 4     21.4  06/01/2021 21:26               20      0.013889
 ...    ...               ...              ...           ...
 1311  20.9  11/01/2021 10:21             6555      4.552083
 1312  21.2  11/01/2021 10:26             6560      4.555556
 1313  21.5  11/01/2021 10:31             6565      4.559028
 1314  21.9  11/01/2021 10:36             6570      4.562500
 1315  22.1  11/01/2021 10:41             6575      4.565972
 
 [1316 rows x 4 columns],
 '125':       cbg               Tid  minutes_elapsed  days_elapsed
 0    12.5  09/11/2020 15:40                0      0.000000
 1    12.3  09/11/2020 15:45                5      0.003472
 2    12.0  09/11/2020 15:50               10 

In [19]:
# Function to find rows where the interval is more than 5 minutes
def find_intervals_more_than_15(data_dict):
    results = {}
    for patient_id, df in data_dict.items():
        # Convert Tid column to datetime
        df_copy = df.copy()
        df_copy['Tid'] = pd.to_datetime(df_copy['Tid'], format='%d/%m/%Y %H:%M')
        
        # Calculate time differences in minutes and align the index
        time_diffs = df_copy['Tid'].diff().dt.total_seconds() / 60
        
        # Find rows where interval is more than 5 minutes
        more_than_15_minutes = df_copy[time_diffs > 15]
        # print("more_than_5_minutes", more_than_5_minutes)
        
        # Store results, if any row has more than 5 minutes interval
        if not more_than_15_minutes.empty:
            results[patient_id] = more_than_15_minutes
        else:
            results[patient_id] = None  # No intervals more than 5 minutes
    return results

# Check for intervals more than 5 minutes for each patient
intervals_more_than_15_results = find_intervals_more_than_15(all_data_dict)

# Output results
for patient_id, df in intervals_more_than_15_results.items():
    if df is not None:
        print(f"Patient {patient_id} has intervals more than 15 minutes at the following timestamps:")
        print(df[['Tid', 'minutes_elapsed']])

In [20]:
all_data_dict

{'187':        cbg               Tid  minutes_elapsed  days_elapsed
 0     22.0  06/01/2021 21:06                0      0.000000
 1     21.9  06/01/2021 21:11                5      0.003472
 2     21.7  06/01/2021 21:16               10      0.006944
 3     21.5  06/01/2021 21:21               15      0.010417
 4     21.4  06/01/2021 21:26               20      0.013889
 ...    ...               ...              ...           ...
 1311  20.9  11/01/2021 10:21             6555      4.552083
 1312  21.2  11/01/2021 10:26             6560      4.555556
 1313  21.5  11/01/2021 10:31             6565      4.559028
 1314  21.9  11/01/2021 10:36             6570      4.562500
 1315  22.1  11/01/2021 10:41             6575      4.565972
 
 [1316 rows x 4 columns],
 '125':       cbg               Tid  minutes_elapsed  days_elapsed
 0    12.5  09/11/2020 15:40                0      0.000000
 1    12.3  09/11/2020 15:45                5      0.003472
 2    12.0  09/11/2020 15:50               10 

In [21]:
def fill_intervals_with_nan(df, time_col='Tid', interval_col='minutes_elapsed', fill_interval=5):
    """
    Fills intervals greater than 15 minutes with rows containing NaN values for every 5-minute interval.
    Args:
    - df (pd.DataFrame): The dataframe with the 'Tid' and 'minutes_elapsed' columns.
    - time_col (str): The name of the time column.
    - interval_col (str): The name of the interval column.
    - fill_interval (int): The interval in minutes to insert rows.
    Returns:
    - pd.DataFrame: The modified dataframe with added rows containing NaN values.
    """
    # Convert Tid to datetime
    df[time_col] = pd.to_datetime(df[time_col], format='%d/%m/%Y %H:%M')
    
    # Calculate time differences in minutes
    df['time_diff'] = df[time_col].diff().dt.total_seconds() / 60
    
    # Loop through the dataframe to find gaps larger than 15 minutes
    new_rows = []
    for i in range(1, len(df)):
        gap = df.loc[i, 'time_diff']
        if gap > 15:
            # Find the start and end times of the gap
            start_time = df.loc[i-1, time_col]
            end_time = df.loc[i, time_col]
            
            # Generate missing timestamps
            missing_times = pd.date_range(start=start_time + pd.Timedelta(minutes=fill_interval),
                                          end=end_time - pd.Timedelta(minutes=fill_interval), 
                                          freq=f'{fill_interval}T')
            
            # Create rows with NaN values for these missing timestamps
            for time in missing_times:
                new_row = {time_col: time, interval_col: None, 'time_diff': None}
                new_rows.append(new_row)

    # Append the missing rows to the original dataframe
    if new_rows:
        new_df = pd.DataFrame(new_rows)
        
        # Filter out columns that are entirely NaN before concatenation
        new_df = new_df.dropna(axis=1, how='all')
        
        # Concatenate the original dataframe with the new rows
        df = pd.concat([df, new_df], ignore_index=True)
        df = df.sort_values(by=time_col).reset_index(drop=True)
        
        # df['Tid'] = df['Tid'].dt.strftime('%d/%m/%Y %H:%M')
        
    # Clean up the temporary columns
    df.drop(columns=['time_diff'], inplace=True)

    return df

def find_intervals_more_than_15_and_fill(data_dict):
    results = {}
    for patient_id, df in data_dict.items():
        # Find rows where intervals are more than 15 minutes
        df_filled = fill_intervals_with_nan(df)
        
        # If no gaps greater than 15 minutes, no filling is needed, just return the original data
        results[patient_id] = df_filled
        
    return results


# Apply the function to fill gaps and add NaNs
filled_data = find_intervals_more_than_15_and_fill(all_data_dict)

# Output the results
for patient_id, df in filled_data.items():
    print(f"Data for patient {patient_id} with filled intervals:")
    print(df)
    # df['Tid'] = df['Tid'].dt.strftime('%d/%m/%Y %H:%M')
    all_data_dict[patient_id] = df


Data for patient 187 with filled intervals:
       cbg                 Tid  minutes_elapsed  days_elapsed
0     22.0 2021-01-06 21:06:00                0      0.000000
1     21.9 2021-01-06 21:11:00                5      0.003472
2     21.7 2021-01-06 21:16:00               10      0.006944
3     21.5 2021-01-06 21:21:00               15      0.010417
4     21.4 2021-01-06 21:26:00               20      0.013889
...    ...                 ...              ...           ...
1311  20.9 2021-01-11 10:21:00             6555      4.552083
1312  21.2 2021-01-11 10:26:00             6560      4.555556
1313  21.5 2021-01-11 10:31:00             6565      4.559028
1314  21.9 2021-01-11 10:36:00             6570      4.562500
1315  22.1 2021-01-11 10:41:00             6575      4.565972

[1316 rows x 4 columns]
Data for patient 125 with filled intervals:
      cbg                 Tid  minutes_elapsed  days_elapsed
0    12.5 2020-11-09 15:40:00                0      0.000000
1    12.3 2020-11-09 

In [22]:
# Output the results
for patient_id, df in all_data_dict.items():
    df['Tid'] = df['Tid'].dt.strftime('%d/%m/%Y %H:%M')
    all_data_dict[patient_id] = df

In [23]:
all_data_dict

{'187':        cbg               Tid  minutes_elapsed  days_elapsed
 0     22.0  06/01/2021 21:06                0      0.000000
 1     21.9  06/01/2021 21:11                5      0.003472
 2     21.7  06/01/2021 21:16               10      0.006944
 3     21.5  06/01/2021 21:21               15      0.010417
 4     21.4  06/01/2021 21:26               20      0.013889
 ...    ...               ...              ...           ...
 1311  20.9  11/01/2021 10:21             6555      4.552083
 1312  21.2  11/01/2021 10:26             6560      4.555556
 1313  21.5  11/01/2021 10:31             6565      4.559028
 1314  21.9  11/01/2021 10:36             6570      4.562500
 1315  22.1  11/01/2021 10:41             6575      4.565972
 
 [1316 rows x 4 columns],
 '125':       cbg               Tid  minutes_elapsed  days_elapsed
 0    12.5  09/11/2020 15:40                0      0.000000
 1    12.3  09/11/2020 15:45                5      0.003472
 2    12.0  09/11/2020 15:50               10 

In [24]:
# Check intervals for each patient
interval_and_days_results = check_intervals_and_count_days(all_data_dict)

# Output results
print(interval_and_days_results)

{'187': 4.565972222222222, '125': 1.6111111111111112, '105': 4.670833333333333, '209': 7.246527777777778, '160': 4.194444444444445, '147': 1.0694444444444444, '102': 2.767361111111111, '195': 5.722916666666666, '123': 4.584027777777778, '158': 8.253472222222221, '145': 1.7986111111111112, '214': 2.0347222222222223, '218': 1.9583333333333333, '215': 2.3506944444444446, '157': 1.3506944444444444, '208': 9.597222222222221, '178': 1.7638888888888888, '188': 6.361111111111111, '131': 7.653472222222222, '153': 3.8541666666666665, '196': 1.0659722222222223, '207': 1.9861111111111112, '189': 5.829861111111111, '116': 0.61875, '173': 6.569444444444445, '221': 2.0416666666666665, '120': 1.1215277777777777, '199': 2.3993055555555554, '201': 6.659722222222222, '200': 6.423611111111111, '140': 1.7534722222222223, '119': 1.65625, '117': 1.5381944444444444, '183': 1.3263888888888888, '184': 2.326388888888889, '213': 6.854166666666667, '210': 1.6527777777777777, '121': 9.51388888888889, '172': 5.72569

In [25]:
# Extract the values
values = list(interval_and_days_results.values())

# Calculate the mean
mean_value = np.mean(values)

# Calculate the standard deviation
std_value = np.std(values)

print(f"Mean: {mean_value}")
print(f"Standard Deviation: {std_value}")

Mean: 4.418472222222222
Standard Deviation: 4.496505134771695


In [26]:
intervals_more_than_15_results = find_intervals_more_than_15(all_data_dict)

In [27]:
intervals_more_than_15_results

{'187': None,
 '125': None,
 '105': None,
 '209': None,
 '160': None,
 '147': None,
 '102': None,
 '195': None,
 '123': None,
 '158': None,
 '145': None,
 '214': None,
 '218': None,
 '215': None,
 '157': None,
 '208': None,
 '178': None,
 '188': None,
 '131': None,
 '153': None,
 '196': None,
 '207': None,
 '189': None,
 '116': None,
 '173': None,
 '221': None,
 '120': None,
 '199': None,
 '201': None,
 '200': None,
 '140': None,
 '119': None,
 '117': None,
 '183': None,
 '184': None,
 '213': None,
 '210': None,
 '121': None,
 '172': None,
 '168': None,
 '167': None,
 '223': None,
 '104': None,
 '170': None,
 '136': None,
 '159': None,
 '151': None,
 '169': None,
 '137': None,
 '118': None,
 '109': None,
 '222': None,
 '165': None,
 '148': None,
 '163': None,
 '192': None,
 '154': None,
 '124': None,
 '150': None,
 '142': None,
 '112': None,
 '135': None,
 '161': None,
 '106': None,
 '114': None,
 '217': None,
 '205': None,
 '138': None,
 '171': None,
 '219': None,
 '141': None,
 '162'